In [1]:
import numpy as np
import pandas as pd
from itertools import islice
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [2]:
%run ../src/acc_df.py

In [3]:
Ac = Acc_data('../data/cleaned/coll_coord.csv')

In [4]:
acc = Ac.get_all_coll() #['']
acc['ds']=acc.INCDTTM.apply(lambda x: pd.to_datetime(x))

In [5]:
import holidays
us_holidays=holidays.UnitedStates()
acc['holiday']=acc.ds.apply(lambda x: (x in us_holidays)*1)

#### Time series by week

In [6]:
acc_04_18 = acc[(acc['year'] != 2019) & (acc['year'] != 2003 )]
acc_w = acc_04_18[['year','week','OBJECTID']].groupby(['year','week']).count()
acc_wh = acc_04_18[['year','week','holiday']].groupby(['year','week']).max()
acc_w = acc_w.join(acc_wh)
len(acc_w)

786

Average for each week in year

In [12]:
week_numbers = acc_w.reset_index()
week_numbers.rename({'OBJECTID': 'acc_nr'}, axis=1, inplace=True)
week_numbers.head(2)

,year,week,acc_nr,holiday
0,2004,1,142,1
1,2004,2,263,0


In [23]:
week_means.iloc[3]

acc_nr    228.2
Name: 4, dtype: float64

In [27]:
week_means=week_numbers[['week','acc_nr']].groupby(['week']).mean()
week_numbers['week_mean']=week_numbers.week.apply(lambda x: week_means['acc_nr'][x])
week_numbers

,year,week,acc_nr,holiday,week_mean
0,2004,1,142,1,240.533333
1,2004,2,263,0,260.800000
2,2004,3,291,0,234.133333
3,2004,4,222,1,228.200000
4,2004,5,277,0,251.133333
5,2004,6,249,0,236.933333
6,2004,7,251,0,231.933333
7,2004,8,236,1,221.266667
8,2004,9,265,0,242.600000
9,2004,10,257,0,245.133333


In [28]:
# average over year
week_numbers.acc_nr.sum()/len(week_numbers)

252.49236641221373

### other data / dummies

In [37]:
week_d = pd.get_dummies(week_numbers.week).join(week_numbers)
week_d['year_red']= week_d.year.apply(lambda x: x-2004)
week_d.drop(['week','year'], axis=1, inplace=True)
# week_d['week_avg']=252.49
week_d.tail()

,1,2,3,4,5,6,7,8,9,10,...,48,49,50,51,52,53,acc_nr,holiday,week_mean,year_red
781,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,261,0,252.466667,14
782,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,274,0,256.400000,14
783,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,275,0,276.600000,14
784,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,234,0,268.133333,14
785,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,150,1,196.066667,14


In [38]:
# X=acc_ds[['year','month','week','day','weekday']]
X = week_d.drop(['acc_nr'], axis=1)
y=week_d['acc_nr']

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [104]:
model = GradientBoostingRegressor(learning_rate=0.001, 
                                  n_estimators=5000, #boosting stages to perform
                                  max_depth=3)
model.fit(X_train, y_train)

y_hat = model.predict(X_test)

In [105]:
model.feature_importances_

array([1.69624161e-02, 8.11928284e-03, 5.50223742e-05, 0.00000000e+00,
       0.00000000e+00, 1.54224531e-04, 0.00000000e+00, 1.69717612e-03,
       1.57649354e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.67514486e-04, 4.20457059e-05, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.21266096e-04, 0.00000000e+00,
       0.00000000e+00, 1.22768664e-03, 5.15860731e-05, 0.00000000e+00,
       0.00000000e+00, 9.52790049e-04, 0.00000000e+00, 1.06314930e-04,
       6.83969724e-05, 0.00000000e+00, 3.19005523e-05, 0.00000000e+00,
       0.00000000e+00, 2.50627798e-03, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.88611890e-04, 2.95916274e-04, 0.00000000e+00,
       4.83224600e-04, 0.00000000e+00, 8.69846741e-04, 4.29706742e-03,
       3.09968954e-05, 1.04037086e-03, 9.47069079e-03, 1.27250524e-03,
       1.21433097e-03, 0.00000000e+00, 4.38731145e-03, 1.91316601e-05,
       1.28323845e-02, 1.62353737e-02, 3.14470385e-01, 5.98951457e-01])

In [106]:
model.score(X_train,y_train)

0.6386324389470801

In [107]:
model.score(X_test,y_test)

0.5806200790025627

In [65]:
model = GradientBoostingRegressor(learning_rate= 0.001, #0.001, 
                                  n_estimators= 5000, #5000, #boosting stages to perform
                                  max_depth=3)
model.fit(X_train, y_train)

y_hat = model.predict(X_test)

In [66]:
model.score(X_train,y_train)

0.6623250781121466

In [67]:
model.score(X_test,y_test)

0.575698706773569

In [108]:
if True:
    grid = {
        'learning_rate': [0.001,0.0001],
        'max_depth': [3,4,5,6],
#         'min_samples_leaf': [1,2,3,4,5],
#         'max_features': ['sqrt',None],
        'n_estimators': [50,100,500,1000,5000],
#         'random_state': [0]
    }
else:  # TEST
    grid = {
    'learning_rate': [1],
    'max_depth': [2], 
    'min_samples_leaf': [2],
#     'max_features': ['sqrt', None],
    'n_estimators': [2],
    'random_state': [0]
}
    

from sklearn.model_selection import GridSearchCV
# confusion_score = make_scorer(confusion_rmse, greater_is_better=False)
gbc_grid_cv = GridSearchCV(
    GradientBoostingRegressor(), 
    grid,
    cv=2,  # number of folds
    return_train_score=True,
    verbose=1, 
    n_jobs=-1)
gbc_grid_cv.fit(X, y)
# Collapse

Fitting 2 folds for each of 40 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   13.4s finished


GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_sampl...=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'learning_rate': [0.001, 0.0001], 'max_depth': [3, 4, 5, 6], 'n_estimators': [50, 100, 500, 1000, 5000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [109]:
gbc_grid_cv.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.001, loss='ls', max_depth=4,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=1000,
             n_iter_no_change=None, presort='auto', random_state=None,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [110]:
gbc_grid_cv.best_score_

0.002564928002457337

In [111]:
pd.DataFrame(gbc_grid_cv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,mean_train_score,std_train_score
0,0.028286,0.001699,0.001620,0.000415,0.001,3,50,"{'learning_rate': 0.001, 'max_depth': 3, 'n_es...",-0.296714,-0.515912,-0.406313,0.109599,29,0.032566,0.050419,0.041492,0.008927
1,0.049232,0.004156,0.001449,0.000147,0.001,3,100,"{'learning_rate': 0.001, 'max_depth': 3, 'n_es...",-0.278601,-0.418166,-0.348384,0.069783,22,0.062480,0.097600,0.080040,0.017560
2,0.292728,0.000634,0.003826,0.000006,0.001,3,500,"{'learning_rate': 0.001, 'max_depth': 3, 'n_es...",-0.235196,0.042849,-0.096173,0.139023,8,0.234001,0.357165,0.295583,0.061582
3,0.661119,0.017734,0.006322,0.000163,0.001,3,1000,"{'learning_rate': 0.001, 'max_depth': 3, 'n_es...",-0.257993,0.234161,-0.011916,0.246077,2,0.352395,0.509369,0.430882,0.078487
4,3.034174,0.031077,0.021783,0.002335,0.001,3,5000,"{'learning_rate': 0.001, 'max_depth': 3, 'n_es...",-0.394186,0.190783,-0.101702,0.292484,10,0.568327,0.707113,0.637720,0.069393
5,0.034066,0.003270,0.001786,0.000059,0.001,4,50,"{'learning_rate': 0.001, 'max_depth': 4, 'n_es...",-0.302457,-0.507174,-0.404815,0.102359,28,0.038248,0.057765,0.048007,0.009759
6,0.082553,0.000382,0.002272,0.000044,0.001,4,100,"{'learning_rate': 0.001, 'max_depth': 4, 'n_es...",-0.287584,-0.405618,-0.346601,0.059017,19,0.072922,0.110163,0.091542,0.018620
7,0.466416,0.019147,0.004187,0.000229,0.001,4,500,"{'learning_rate': 0.001, 'max_depth': 4, 'n_es...",-0.261396,0.104754,-0.078321,0.183075,5,0.281466,0.398309,0.339888,0.058421
8,0.946202,0.026689,0.007868,0.000561,0.001,4,1000,"{'learning_rate': 0.001, 'max_depth': 4, 'n_es...",-0.266231,0.271360,0.002565,0.268796,1,0.419000,0.564233,0.491616,0.072617
9,3.647011,0.113586,0.019526,0.000588,0.001,4,5000,"{'learning_rate': 0.001, 'max_depth': 4, 'n_es...",-0.481588,0.154313,-0.163637,0.317951,14,0.659658,0.776814,0.718236,0.058578
